## SONAR: Emotionally Intelligent Music Recommendations

Draws heavily from [this tutorial](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html)

Welcome to Sonar! This project was inpired me to first explore Machine Learning. The premise is simple: we often want different music to listen to based on our mood. However, oftentimes finding that music proves difficult. Sonar takes away that difficulty: by entering short mood messages, sonar generates Spotify recommendations using both your old favorites and determined mood. 

Enjoy!

In [1]:
import re

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, BoxSelectTool

import gensim
from gensim.models.word2vec import Word2Vec
from keras import Sequential
from keras.layers import Dense
import numpy as np
from nltk.tokenize import TweetTokenizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import spacy
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook, tqdm

nlp = spacy.load("en")
tqdm_notebook().pandas(desc="progress-bar")
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
TaggedDocument = gensim.models.doc2vec.TaggedDocument

Using TensorFlow backend.


### Preprocessing

- Load tweets into a usable format. Binarize sentiment from {0, 1, 2, 3, 4} to {0, 1} with 2 serving as the cut off for a "positive" sentiment. Clean up null values and reindex.

- Tokenize the data and take only the n values we aim to use in training.

In [2]:
def load_tweets():
    tweets_train = pd.read_csv("/home/malits/data/twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv", 
                               encoding = "ISO-8859-1")
    data = pd.DataFrame()
    data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v >= 2) else 0)
    data["sentiment"] = data["sentiment"].map(int)
    data["tweet"] = tweets_train.iloc[:, [5]]
    data["tweet"] = data["tweet"].astype(str)
    data = data[data['tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    
    return data

In [3]:
data = load_tweets()
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [4]:
def tokenize(tweet):
    sub_regex = r'(#[A-Za-z0-9])|(https)\?!\.,'
    tweet = str(tweet)
    tokens = tokenizer.tokenize(tweet)
    tokens = [re.sub(sub_regex, '', t) for t in tokens]
    tokens = [t.lower() for t in tokens if t is not None and t != '']
    return tokens

In [5]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data.loc[:, "tokens"] = data.loc[:, "tweet"].progress_apply(tokenize)
    data = data.loc[data.tokens != 'NC', :]
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [6]:
processed = postprocess(data)

/home/malits/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/malits/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Create training data

Once preprocessed, split to test/train sets and vectorize w w2vec for use in keras

In [7]:
# Size of our embedding space

N_DIM = 200

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(processed['tokens']),
                                                    np.array(processed['sentiment']),
                                                    test_size=0.2)

In [9]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i, v in tqdm_notebook(enumerate(tweets)):
        label = '{}_{}'.format(label_type, i)
        ## Use nltk TaggedDocument obj to mark words with their corresponding label. Makes words easier to retrieve from their vector form.
        labelized.append(TaggedDocument(v, [label]))
    return labelized

In [10]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')

## Replace Word2Vec corpus with more data!

In [11]:
train_size = len(X_train)
tweet_w2v = Word2Vec(size=N_DIM, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm_notebook(X_train)])
tweet_w2v.train([x.words for x in tqdm_notebook(X_train)], 
                total_examples=train_size, epochs = tweet_w2v.epochs)

(43227464, 60104860)

In [52]:
def get_similarity(text):
    return tweet_w2v.wv.most_similar(text.lower())

In [79]:
get_similarity('kanye')

[('echo', 0.6594349145889282),
 ('indies', 0.6447641253471375),
 ('alexander', 0.6290473937988281),
 ('undead', 0.6146681904792786),
 ('cattle', 0.6079335808753967),
 ('shore', 0.6066163182258606),
 ('collective', 0.602022647857666),
 ('brom', 0.5925573110580444),
 ('ford', 0.5919807553291321),
 ('african', 0.5899761915206909)]

## Transforming

We use TF-IDF weightings to weight word vectors accordingly

In [14]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [80]:
def create_tweet_vector(words, ndims):
    vector = np.zeros((1, ndims))
    count = 0
    
    for w in words:
        try:
            vector += tweet_w2v.wv[w].reshape((1, ndims)) * tfidf[w]
            count += 1
        except KeyError:
            continue
            
    return np.divide(vector,count)

In [81]:
train_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_train))])
train_vec_tweets = scale(train_vec_tweets)

test_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_test))])
test_vec_tweets = scale(test_vec_tweets)

/home/malits/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


/home/malits/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


## Neural Architecture in Keras

In [82]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(train_vec_tweets, y_train)

Epoch 1/1
800000/800000 [==============================] - 29s 36us/step - loss: nan - acc: 1.7750e-04


In [35]:
model.evaluate(test_vec_tweets, y_test)

200000/200000 [==============================] - 3s 13us/step


[nan, 0.0]

In [41]:
test_vec_tweets

array([[-0.10839082,  0.96090095,  1.19466249, ...,  0.04442698,
        -2.70746003, -0.36207966],
       [ 0.49944944,  0.6660377 ,  1.02380939, ..., -0.57288695,
        -0.27184162, -0.02124841],
       [ 0.25561972, -0.33809096,  0.32096454, ..., -0.57700449,
         0.89326356,  0.82663036],
       ...,
       [ 0.1258103 ,  0.53280072,  0.28238178, ...,  0.70347905,
        -0.40420087,  0.21112377],
       [ 1.8908536 ,  1.22318712, -1.38479328, ...,  0.73263885,
        -0.0792803 , -1.56691066],
       [-0.90170334,  0.2833283 ,  1.54325774, ..., -3.53057298,
         3.5072139 , -1.53493829]])